# T1547.010 - Port Monitors
Adversaries may use port monitors to run an adversary supplied DLL during system boot for persistence or privilege escalation. A port monitor can be set through the <code>AddMonitor</code> API call to set a DLL to be loaded at startup.(Citation: AddMonitor) This DLL can be located in <code>C:\Windows\System32</code> and will be loaded by the print spooler service, spoolsv.exe, on boot. The spoolsv.exe process also runs under SYSTEM level permissions.(Citation: Bloxham) Alternatively, an arbitrary DLL can be loaded if permissions allow writing a fully-qualified pathname for that DLL to <code>HKLM\SYSTEM\CurrentControlSet\Control\Print\Monitors</code>. 

The Registry key contains entries for the following:

* Local Port
* Standard TCP/IP Port
* USB Monitor
* WSD Port

Adversaries can use this technique to load malicious code at startup that will persist on system reboot and execute as SYSTEM.

## Atomic Tests

### Atomic Test #1 - Add Port Monitor persistence in RegistryAdd key-value pair to a Windows Port Monitor registry. On the subsequent reboot dll will be execute under spoolsv with NT AUTHORITY/SYSTEM privilege.**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "hklm\system\currentcontrolset\control\print\monitors\ART" /v "Atomic Red Team" /d "C:\Path\AtomicRedTeam.dll" /t REG_SZ
```

In [ ]:
Invoke-AtomicTest T1547.010 -TestNumbers 1

#### Cleanup: 
```cmd
reg delete "hklm\system\currentcontrolset\control\print\monitors\ART" /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1547.010 -TestNumbers 1 -Cleanup

## Detection
Monitor process API calls to <code>AddMonitor</code>.(Citation: AddMonitor) Monitor DLLs that are loaded by spoolsv.exe for DLLs that are abnormal. New DLLs written to the System32 directory that do not correlate with known good software or patching may be suspicious. 

Monitor Registry writes to <code>HKLM\SYSTEM\CurrentControlSet\Control\Print\Monitors</code>. Run the Autoruns utility, which checks for this Registry key as a persistence mechanism.(Citation: TechNet Autoruns)